<hr style="border-width:2px;border-color:#75DFC1">
<h1 style = "text-align:center">Time Series Forecasting</h1>
<h2 style = "text-align:center">Univariate time series - Complex study case (no straightforward seasonality/cyclic behaviour, missing dates) - Stock market prediction </h2>

<h2 style = "text-align:center">LSTM</h2>
<h4 style = "text-align:center">Didier Law-Hine</h4>
<h4 style = "text-align:center">August 2022</h4>

<hr style="border-width:2px;border-color:#75DFC1">

In this notebook, the dataset consists of records about the stock price of Tata Global Beverages Limited. The dataset also contains a date-wise price of stock with open, close, high, and low prices along with volume traded as well as turnover on that day.

For LSTM model, I consider the following important parameters to be tuned :
- the number of lag $n$ to account for the prediction of  $y_t$ :  $y_{t-1},y_{t-2}, ... , y_{t-n}$. In this notebook, you will see $n$=30.
- number of hidden layers and units in each layer. Here, I chose 1 layer of 50 units.
- epochs and batch size. Here, typically epochs=80, batch size=20

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from math import sqrt
from numpy import concatenate
from matplotlib.pylab import rcParams
rcParams['figure.figsize']=20,8

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error

import statsmodels.api as sm
import statsmodels.graphics.tsaplots as smgt

from keras.models import Sequential
from keras.layers import LSTM,Dropout,Dense

from pmdarima.arima import auto_arima
from sktime.forecasting.arima import AutoARIMA

In [ ]:
# Load dataset
df = pd.read_csv("NSE-Tata-Global-Beverages-Limited.csv")
df["Date"]=pd.to_datetime(df.Date,format="%Y-%m-%d")
display(df.head())
display(df.info())
# Dataviz
fig,ax1=plt.subplots()
ax1=plt.plot(df["Date"],df["Close"],label='Close Price history')

In [ ]:
# Selecting the time series from the initial dataframe 
df2=df.sort_values(by=["Date"],ascending=True,axis=0).reset_index(drop=True)
df3=pd.DataFrame(index=range(0,len(df2)),columns=['Date','Close'])
for i in range(0,len(df3)):
    df3["Date"][i]=df2['Date'][i]
    df3["Close"][i]=df2["Close"][i]   
df3.index=df3.Date
df3.drop("Date",axis=1,inplace=True)

# Resampling
df3=df3.asfreq(freq='D') # This will add rows so that frequency is equal to 1 day. But this will generate NaNs
display(df3.head(10))
display(df3.shape)
# Filling NaNs with interpolating
#https://towardsdatascience.com/4-techniques-to-handle-missing-values-in-time-series-data-c3568589b5a8
df3=df3.astype('float')
df3['Close'].interpolate(inplace=True,limit_direction='forward', axis=0) #default is linear interpolating
display(df3.head(10))
display(df3.shape)

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
res = seasonal_decompose(df3,freq=52)
res.plot()
plt.show()

In [ ]:
from statsmodels.tsa.seasonal import STL

stl = STL(df3, robust=True,period=52)
res = stl.fit()
fig = res.plot()

In [ ]:
def tsplot(y, lags=None, figsize=(12, 7)):
    """
        Plot time series, its ACF and PACF, calculate ADF and KPSS tests
        y - timeseries
        lags - how many lags to include in ACF, PACF calculation
    """
    if not isinstance(y, pd.Series):
        y = pd.Series(y)
           
    fig = plt.figure(figsize=figsize)
    layout = (2, 2)
    ts_ax = plt.subplot2grid(layout, (0, 0), colspan=2)
    acf_ax = plt.subplot2grid(layout, (1, 0))
    pacf_ax = plt.subplot2grid(layout, (1, 1))

    y.plot(ax=ts_ax)
    p_value_ADF = sm.tsa.stattools.adfuller(y)[1]
    p_value_KPSS = sm.tsa.stattools.kpss(y)[1]
    ts_ax.set_title('Time Series Analysis Plots\n Dickey-Fuller: p={0:.5f} \n KPSS: p={1:.5f}'.format(p_value_ADF,p_value_KPSS))
    sm.graphics.tsa.plot_acf(y, lags=lags, ax=acf_ax)
    sm.graphics.tsa.plot_pacf(y, lags=lags, ax=pacf_ax)
    plt.tight_layout()

# Plot
tsplot(df3["Close"])

In [ ]:
ts_diff = (df3["Close"] - df3["Close"].shift(1)).dropna()
tsplot(ts_diff)

In [ ]:
model=sm.tsa.SARIMAX(df3[0:1461],order=(1,1,0),seasonal_order=(1,2,0,12))
sarima=model.fit()
print(sarima.summary())

In [ ]:
from datetime import datetime, timedelta
t1 = datetime(2018,1,1)
t2 = datetime(2018,12,20)
pred = sarima.predict(t1,t2)
concatpred = pd.concat([df3, pred])#Concaténation des prédictions

plt.plot(concatpred) #Visualisation


In [ ]:
print(t1)
print(df3.iloc[1234])

In [ ]:
df3.shape

In [ ]:
df3.info()

In [ ]:
1827*0.8

In [ ]:
from pmdarima.arima import auto_arima
train=df3[0:1461].astype(float)
test=df3[1461:1827].astype(float)
model = auto_arima(y=train,D=1,d=1, error_action='ignore', suppress_warnings=True)
model.fit(train)
display(model.summary())
forecast = model.predict(n_periods=len(test))
forecast = pd.DataFrame(forecast,index = test.index,columns=['Prediction'])

#plot the predictions for validation set
plt.plot(train, label='Train')
plt.plot(test, label='Valid')
plt.plot(forecast, label='Prediction')
plt.show()

In [ ]:
train=df3[0:1461].astype(float)
test=df3[1461:1827].astype(float)

# Fit two different ARIMAs
m7 = auto_arima(train, error_action='ignore', seasonal=True,d=1, m=7)
m12 = auto_arima(train, error_action='ignore', seasonal=True, d=1, m=52)

fig, axes = plt.subplots(1, 2, figsize=(12, 8))
x = np.arange(test.shape[0])

# Plot m=1
axes[0].scatter(x, test, marker='x')
axes[0].plot(x, m7.predict(n_periods=test.shape[0]))
axes[0].set_title('Test samples vs. forecasts (m=7)')

# Plot m=12
axes[1].scatter(x, test, marker='x')
axes[1].plot(x, m12.predict(n_periods=test.shape[0]))
axes[1].set_title('Test samples vs. forecasts (m=12)')


In [ ]:
from sktime.forecasting.arima import AutoARIMA
train=df3[0:1461].astype(float)
test=df3[1461:1827].astype(float)
forecaster = AutoARIMA(D=1,start_p=1, max_p=2, suppress_warnings=True)
train.index = train.index.astype(int)
forecaster.fit(train)
forecaster.summary()

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error

def plot_forecast(series_train, series_test, forecast, forecast_int=None):

    mae = mean_absolute_error(series_test, forecast)
    mape = mean_absolute_percentage_error(series_test, forecast)

    plt.figure(figsize=(12, 6))
    plt.title(f"MAE: {mae:.2f}, MAPE: {mape:.3f}", size=18)
    series_train.plot(label="train", color="b")
    series_test.plot(label="test", color="g")
    forecast.index = series_test.index
    forecast.plot(label="forecast", color="r")
    if forecast_int is not None:
        plt.fill_between(
            series_test.index,
            forecast_int["lower"],
            forecast_int["upper"],
            alpha=0.2,
            color="dimgray",
        )
    plt.legend(prop={"size": 16})
    plt.show()

    return mae, mape


t1 = datetime(2018,1,1)
t2 = datetime(2018,12,20)
fh =pd.to_datetime(np.arange(t1, t2, timedelta(days=1)).astype(datetime))
X=test
forecast, forecast_int = forecaster.predict([fh, X])
sun_arima_mae, sun_arima_mape = plot_forecast(
    train, test, forecast, forecast_int
)

In [ ]:
fh

In [ ]:
## Preparing data for LSTM
# Convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

# Selecting the time series from the initial dataframe --> called 'values' (np.array)
values=df3.values
display(values.shape)
# integer encode direction
##encoder = LabelEncoder()
##values[:,4] = encoder.fit_transform(values[:,4])
# ensure all data is float
values = values.astype('float32')
# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
# specify the number of lag hours
n_days = 30
n_features = 1
# frame as supervised learning
reframed = series_to_supervised(scaled, n_days, 1)
print(reframed.head())
print(reframed.shape)

# split into train and test sets
values = reframed.values
n_train_days = 987
train = values[:n_train_days, :]
test = values[n_train_days:, :]
# split into input and outputs
n_obs = n_days * n_features
X_train, y_train = train[:, :n_obs], train[:, -n_features]
X_test, y_test = test[:, :n_obs], test[:, -n_features]
print(X_train.shape, len(X_train), y_train.shape)
# reshape input to be 3D [samples, timesteps, features]
X_train = X_train.reshape((X_train.shape[0], n_days, n_features))
X_test = X_test.reshape((X_test.shape[0], n_days, n_features))
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

In [ ]:
## Model : LSTM(50),Dense(1)
# design network
model = Sequential()
model.add(LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam')
# fit network
history = model.fit(X_train, y_train, epochs=80, batch_size=20, validation_data=(X_test, y_test), verbose=2, shuffle=False)
# plot history
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()

In [ ]:
# Make a prediction
yhat = model.predict(X_test)
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1]))
# invert scaling for forecast
inv_yhat = concatenate((yhat, X_test[:, - n_features -1:]), axis=1)
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:,0]
# invert scaling for actual
y_test = y_test.reshape((len(y_test), 1))
inv_y = concatenate((y_test, X_test[:, - n_features -1:]), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]

# calculate RMSE
rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test RMSE: %.3f' % rmse)

# Plot prediction
df_values=df3.Close
d = {'train': df_values, 'test': pd.Series(inv_yhat, index=df_values.index[np.arange(n_train_days+n_days,len(df_values))])}
df_plot=pd.DataFrame(data=d, index=df3.index)
plt.figure();
plt.plot(df_plot);
